In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
users_data = pd.read_csv("users_data.csv")
users_data.shape

(3214, 5)

In [3]:
users_data.columns

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

In [4]:
users_data.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [6]:
users_data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

### There are no missed values

In [8]:
users_data.duplicated().sum()

0

### There are no duplicated rows

## Splitting raw data by samples

In [9]:
features = users_data.drop(['is_ultra'], axis=1)
target = users_data['is_ultra']
features.shape, target.shape

((3214, 4), (3214,))

In [13]:
train_features, valid_features, train_target, valid_target = train_test_split(features, target, test_size=0.20,
                                                                              random_state=42, stratify=target)
train_features, test_features, train_target, test_target = train_test_split(train_features, train_target, test_size=0.25,
                                                                            random_state=42, stratify=train_target)

In [16]:
print("TRAIN sample shape: {:2.2%}".format(train_features.shape[0]/features.shape[0]))
print("VALID sample shape: {:2.2%}".format(valid_features.shape[0]/features.shape[0]))
print("TEST sample shape: {:2.2%}".format(test_features.shape[0]/features.shape[0]))

TRAIN sample shape: 59.99%
VALID sample shape: 20.01%
TEST sample shape: 20.01%
